# MPRA training (for analyses)

This notebook process the MPRA results to generate the table used to produce the analyses reported in the CODA paper. A slightly different table is used to optimize Malinois weights, which can be created using the notebook"CODA_preprocess_MPRA_validation"

In [1]:
import os
import numpy as np
import pandas as pd
import gzip

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

pd.set_option('display.max_columns', None)

/var/folders/sq/pxxh686s20scbcsrvq340czj6yvqp0/T/ipykernel_44951/2667858688.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Fetch the MPRA results

The dataset consists of oligo libraries (OL) from the MPRA projects UKBB (OL27-33), GTEx (OL41,42), and CRE (OL15).

In [2]:
rootdir = '/training'

cell_types_lower = ['k562', 'hepg2', 'sknsh']
cell_types = ['K562', 'HepG2', 'SKNSH']
std_cell_names = dict(zip(cell_types_lower, cell_types))

UKBB_libraries = [f'OL{n}' for n in range(27, 34)]
GTEx_libraries = ['OL41', 'OL42', 'OL41-42', 'OL41B']
CREs_libraries = ['OL15']

data_project_dict = {OL: 'UKBB' for OL in UKBB_libraries}
data_project_dict.update({OL: 'GTEx' for OL in GTEx_libraries})
data_project_dict.update({OL: 'CRE' for OL in CREs_libraries})

print('Fetching files:\n')
in_df = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".out") and ('Counts' not in filepath):
            library_num, cell_type = file.split('_')[:2]
            if cell_type.lower() in cell_types_lower:
                df_temp = pd.read_csv(filepath, sep='\t', low_memory=False)
                df_temp['OL'] = library_num
                df_temp['cell_type'] = std_cell_names[cell_type.lower()]
                df_temp['data_project'] = data_project_dict[library_num]
                in_df.append(df_temp)
                print(file)

in_columns = ['ID', 'chr', 'project', 'ctrl_exp', 'Ctrl.Mean',
       'Exp.Mean', 'log2FoldChange', 'lfcSE', 'OL',
       'cell_type', 'data_project']

in_df = pd.concat(in_df)[in_columns]

row_filter = in_df['chr'].isna()
chrs = []
for ID in in_df[row_filter]['ID'].tolist():
    if 'sample' in ID:
        chrs.append('synth')
    else:
        chrs.append(ID.split(':')[0])
        
in_df.loc[row_filter, 'chr'] = chrs

in_df.loc[in_df['project'].isna(), 'project'] = ''

Fetching files:

OL42_HepG2_20211010.out
OL28_SKNSH_20210428.out
OL41_SKNSH_20211010.out
OL42_SKNSH_20211010.out
OL28_HEPG2_20210428.out
OL27_K562_20210428.out
OL33_HEPG2_20210428.out
OL15_HEPG2_Neon_20200904.out
OL30_SKNSH_20210428.out
OL33_SKNSH_20210428.out
OL41B_HepG2_20211010.out
OL30_HEPG2_20210428.out
OL32_SKNSH_20210428.out
OL33_K562_20210428.out
OL15_K562_20200904.out
OL31_HEPG2_20210428.out
OL15_SKNSH_20200904.out
OL32_HEPG2_20210428.out
OL32_K562_20210428.out
OL31_SKNSH_20210428.out
OL27_SKNSH_20210428.out
OL28_K562_20210428.out
OL31_K562_20210428.out
OL29_HEPG2_20210428.out
OL27_HEPG2_20210428.out
OL29_K562_20210428.out
OL30_K562_20210428.out
OL29_SKNSH_20210428.out
OL41-42_K562_20211010.out


### Filter oligos with no RNA count

Before aggregating across libraries, we fitler out sequences with no RNA count.

Note: In order to reproduce the table used to optimize Malinois weights one would need to also filter out sequences with an oligo count less than 20.

In [3]:
oligo_count_cutoff = 0
RNA_count_cutoff = 0

quality_filter = (in_df['Ctrl.Mean'] >= oligo_count_cutoff) & \
                         (in_df['Exp.Mean'] > RNA_count_cutoff)

in_df = in_df[quality_filter].reset_index(drop=True) 

### Get the nucleotide sequence

We load the nuecleotide sequence information from each corresponding fasta file and assign to each oligo ID its corresponding sequence. It is worth noting that there might be very rare edge cases in which the same oligo ID might differ by one position between the UKBB and GTEx databases. To our knowledge, there is only one case. Nonetheless, fetching the sequence from the corresponding OL fasta file (as opposed to just creating a set of ID-to_sequence pairs) ensures we get the correct measured sequence.

In [4]:
rootdir = '/Fastas'

ignore_libraries = ['OL46']

fasta_dict = {'UKBB': {}, 'GTEx': {}, 'CRE': {}}
print('Processing fasta files:\n')
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".gz") and ('fasta' in filepath):
            library_num = file.split('_')[0]
            if library_num not in ignore_libraries:
                print(file)
                project = data_project_dict[library_num]
                with gzip.open(filepath, 'rt') as f:
                    for line_str in f:
                        if line_str[0] == '>':
                            oligo_id = line_str.lstrip('>').rstrip('\n')
                            fasta_dict[project][oligo_id] = ''
                        else:
                            fasta_dict[project][oligo_id] += line_str.rstrip('\n')

for project in fasta_dict.keys():
    project_fasta_dict = fasta_dict[project]
    compressed_IDs = [ID for ID in project_fasta_dict.keys() if ';' in ID]
    for compressed_ID in compressed_IDs:
        stripped_ID = compressed_ID.lstrip('(').rstrip(')')
        for single_ID in stripped_ID.split(';'):
            project_fasta_dict[single_ID] = project_fasta_dict[compressed_ID]
        
in_df['sequence'] = in_df.apply(lambda x: fasta_dict[x['data_project']][x['ID']], axis=1)

Processing fasta files:

OL41_reference.fasta.gz
OL32_reference.fasta.gz
OL31_reference.fasta.gz
OL42_reference.fasta.gz
OL27_reference.fasta.gz
OL29_reference.fasta.gz
OL15_reference.fasta.gz
OL28_reference.fasta.gz
OL30_reference.fasta.gz
OL33_reference.fasta.gz


### Grouping results for oligos shared across libraries in a data project

This step is needed only for sequences in the UKBB oligo libraries, since these libraries share some sequences. However it does hurt to do it for the libraries in GTEx and CRE. For practical purposes, we average the log2(Fold-Change) readout of sequences shared across libraries in order to have a single sequence-to-activity pair for each sequence. For measures that can be inform us of empirical noise (such as oligo count, RNA count, and lof2FC standard error), we retrieve the "worst" value (min, min, max, respectively) across libraries.

Note: For legacy reasons, we do not average shared sequences across projects since the generation of these projects was asynchronous. However, the consistency of the MPRA results probably allows for averaging across different projects as well.

In [5]:
grouped_project_dfs = []
for data_project in ['UKBB', 'GTEx', 'CRE']:
    print(f'Grouping sequences across cell types and across libraries in {data_project}')
    project_df = in_df[in_df['data_project'] == data_project].reset_index(drop=True)
    groups = project_df.groupby(by=['sequence', 'cell_type']).agg(
                log2FC_mean = ('log2FoldChange', 'mean'),
                chromosome = ('chr', 'first'),
                OL = ('OL', set),
                classes = ('project', set),
                IDs = ('ID', set),
                exp_mean = ('Exp.Mean', 'min'),
                ctrl_mean = ('Ctrl.Mean', 'min'),
                lfcSE = ('lfcSE', 'max'))
    grouped_project_df = groups.unstack(level=-1)
    grouped_project_df.columns = ['_'.join(col).rstrip('_') for col in grouped_project_df.columns.values]
    grouped_project_df.reset_index(inplace=True)
    grouped_project_df['data_project'] = data_project
    grouped_project_dfs.append(grouped_project_df)

Grouping sequences across cell types and across libraries in UKBB
Grouping sequences across cell types and across libraries in GTEx
Grouping sequences across cell types and across libraries in CRE


### Clean the grouped columns

These lines just clean up the columns generated from stacking the groups generated in the previous cell.

In [6]:
column_drop_list = ['chromosome_K562', 'chromosome_SKNSH',
               'classes_K562','classes_SKNSH',
               'IDs_K562', 'IDs_SKNSH',
               'OL_K562', 'OL_HepG2',
              ]

new_column_name_dict = {'sequence': 'sequence',
                    'log2FC_mean_HepG2': 'HepG2_log2FC',
                    'log2FC_mean_K562': 'K562_log2FC',
                    'log2FC_mean_SKNSH': 'SKNSH_log2FC',
                    'chromosome_HepG2': 'chr',
                    'classes_HepG2': 'class',
                    'IDs_HepG2': 'IDs',
                    'OL_SKNSH': 'OL',
                    'exp_mean_HepG2': 'exp_mean_HepG2',
                    'exp_mean_K562': 'exp_mean_K562',
                    'exp_mean_SKNSH': 'exp_mean_SKNSH',
                    'ctrl_mean_HepG2': 'ctrl_mean_HepG2',
                    'ctrl_mean_K562': 'ctrl_mean_K562',
                    'ctrl_mean_SKNSH': 'ctrl_mean_SKNSH',
                    'lfcSE_HepG2': 'HepG2_lfcSE',
                    'lfcSE_K562': 'K562_lfcSE',
                    'lfcSE_SKNSH': 'SKNSH_lfcSE',
                    'data_project': 'data_project',
                   }

out_column_order = ['IDs', 'sequence', 'data_project', 'OL', 'class', 'chr',
                    'K562_log2FC', 'HepG2_log2FC', 'SKNSH_log2FC',
                    'K562_lfcSE', 'SKNSH_lfcSE', 'HepG2_lfcSE', 
#                     'ctrl_mean_K562', 'ctrl_mean_HepG2', 'ctrl_mean_SKNSH',
#                     'exp_mean_K562', 'exp_mean_HepG2', 'exp_mean_SKNSH',
                   ]

out_df = pd.concat(grouped_project_dfs)

out_df.drop(column_drop_list, axis=1, inplace=True)
out_df.columns = [new_column_name_dict[old_name] for old_name in out_df.columns.values]

null_filter = (out_df['HepG2_log2FC'].notnull()) & (out_df['K562_log2FC'].notnull()) & (out_df['SKNSH_log2FC'].notnull())
out_df = out_df[null_filter].reset_index(drop=True)

out_df['OL'] = out_df['OL'].apply(lambda x: ','.join(sorted(x)))
out_df['class'] = out_df['class'].apply(lambda x: ','.join(sorted(x)))
out_df['IDs'] = out_df['IDs'].apply(lambda x: ';'.join(sorted(x)))

compressed_ids_filter = out_df['IDs'].str.contains(';')
out_df.loc[compressed_ids_filter, 'IDs'] = out_df[compressed_ids_filter]['IDs'].apply(lambda x: '(' + ';'.join(x.split(';')) + ')')

out_df = out_df[out_column_order]

### Drop duplicated sequences across projects

In order to keep a single sequence-to-activity pair for each sequence, we drop duplicated oligos across projects. If a duplicated oligo is present in UKBB, we keep only the UKBB information. If a duplicated oligo is present in GTEx and CRE but not UKBB, we keep only the GTEx information.

In [7]:
out_df = out_df.sort_values(by=['data_project'], key=lambda x: x.map({'UKBB': 0, 'GTEx': 1, 'CRE':2})).reset_index(drop=True)
out_df = out_df.drop_duplicates('sequence', keep='first').reset_index(drop=True)

out_df = out_df[out_df['chr'] != 'synth'].reset_index(drop=True)

out_df

,IDs,sequence,data_project,OL,class,chr,K562_log2FC,HepG2_log2FC,SKNSH_log2FC,K562_lfcSE,SKNSH_lfcSE,HepG2_lfcSE
0,7:89640175:A:G:A:wC,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGAATGTCAGTT...,UKBB,OL29,BMI,7,0.955879,0.292685,-0.780102,0.559413,2.904046,1.148763
1,6:159961451:G:A:A:wC,GGTGTCAGAACGCCCCAGGGGGCTTTGAGGGCTGCTGCCCCTGGAG...,UKBB,OL30,SBP,6,2.059735,0.986049,-0.431290,0.097276,0.089045,0.087488
2,18:41953230:C:T:A:wC,GGTGTCACAGTAATGTGGAGGAGTAGAGGCACTCTGGCTTTTTGAG...,UKBB,OL27,Plt,18,0.256503,0.266996,-0.177992,0.088237,0.111069,0.100591
3,18:41953230:C:T:R:wC,GGTGTCACAGTAATGTGGAGGAGTAGAGGCACTCTGGCTTTTTGAG...,UKBB,OL27,Plt,18,0.353888,0.132264,-0.052014,0.080057,0.105239,0.100011
4,21:43635174:A:G:A:wC,GGTGTCAAGTGAGCGAGATGTCCCCGGCCACACAGTGAAGCACAAG...,UKBB,OL32,Age_at_Menarche,21,-0.174600,-0.422087,-0.416915,0.184045,0.168331,0.212965
...,...,...,...,...,...,...,...,...,...,...,...,...
798059,18:9125893:NA:NA,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...,CRE,OL15,,18,-0.204913,-0.156933,-0.209358,0.133052,0.185115,0.157279
798060,12:33905808:NA:NA,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...,CRE,OL15,,12,1.218233,0.613623,0.569894,0.127132,0.190639,0.167222
798061,3:128145854:NA:NA,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...,CRE,OL15,,3,-0.222234,-0.338764,-0.817852,0.159002,0.238637,0.198187
798062,8:134705703:NA:NA,CAGGTTGAAGGGAATCCAAGGGCCATTTCCTTGGGAATTTCAGAAA...,CRE,OL15,,8,-0.194952,-0.199087,0.169847,0.131227,0.170301,0.152272


### Optional log2FC standard error filter
We usually apply the log2FC standard error filter below when we analize MPRA activity and/or compare it to model predictions.

In [8]:
temp_df = out_df.copy()
stderr_columns = ['K562_lfcSE', 'HepG2_lfcSE', 'SKNSH_lfcSE']
stderr_threshold = 1

quality_filter = temp_df[stderr_columns].max(axis=1) < stderr_threshold
temp_df = temp_df[quality_filter].reset_index(drop=True)
temp_df

,IDs,sequence,data_project,OL,class,chr,K562_log2FC,HepG2_log2FC,SKNSH_log2FC,K562_lfcSE,SKNSH_lfcSE,HepG2_lfcSE
0,6:159961451:G:A:A:wC,GGTGTCAGAACGCCCCAGGGGGCTTTGAGGGCTGCTGCCCCTGGAG...,UKBB,OL30,SBP,6,2.059735,0.986049,-0.431290,0.097276,0.089045,0.087488
1,18:41953230:C:T:A:wC,GGTGTCACAGTAATGTGGAGGAGTAGAGGCACTCTGGCTTTTTGAG...,UKBB,OL27,Plt,18,0.256503,0.266996,-0.177992,0.088237,0.111069,0.100591
2,18:41953230:C:T:R:wC,GGTGTCACAGTAATGTGGAGGAGTAGAGGCACTCTGGCTTTTTGAG...,UKBB,OL27,Plt,18,0.353888,0.132264,-0.052014,0.080057,0.105239,0.100011
3,21:43635174:A:G:A:wC,GGTGTCAAGTGAGCGAGATGTCCCCGGCCACACAGTGAAGCACAAG...,UKBB,OL32,Age_at_Menarche,21,-0.174600,-0.422087,-0.416915,0.184045,0.168331,0.212965
4,21:43635174:A:G:R:wC,GGTGTCAAGTGAGCGAGATGTCCCCGGCCACACAGTGAAGCACAAG...,UKBB,OL32,Age_at_Menarche,21,0.081310,-0.369311,-0.377099,0.152207,0.167750,0.237189
...,...,...,...,...,...,...,...,...,...,...,...,...
749049,18:9125893:NA:NA,CAGTACTGCTGGCCCCAGAAAAGCCCCTCTCCTTATACCCTAGGCC...,CRE,OL15,,18,-0.204913,-0.156933,-0.209358,0.133052,0.185115,0.157279
749050,12:33905808:NA:NA,CAGTACCTTGTCCCCACTTCCCATTTGGCCTCTGGCAGAGGAGGAG...,CRE,OL15,,12,1.218233,0.613623,0.569894,0.127132,0.190639,0.167222
749051,3:128145854:NA:NA,CAGTACACCCCAGCTTCCAAAGGCCTTCTGTGACAAAGAGAGACTA...,CRE,OL15,,3,-0.222234,-0.338764,-0.817852,0.159002,0.238637,0.198187
749052,8:134705703:NA:NA,CAGGTTGAAGGGAATCCAAGGGCCATTTCCTTGGGAATTTCAGAAA...,CRE,OL15,,8,-0.194952,-0.199087,0.169847,0.131227,0.170301,0.152272
